## 1. Importar módulos necesarios

In [1]:
# ## 1. Importar módulos necesarios

import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.utils import to_categorical

2025-02-05 12:26:21.162539: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-05 12:26:21.228957: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-05 12:26:21.305429: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738754781.368225   12188 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738754781.384129   12188 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-05 12:26:21.536427: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

## 1.1 Comprobar si los paquetes se han cargado correctamente

In [2]:
# ## 1.1 Comprobar si los paquetes se han cargado correctamente

from tensorflow.keras import Input

print("TensorFlow version:", tf.__version__)
print("Eager execution:", tf.executing_eagerly())

# Verificar si Keras funciona
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Input(shape=(20,)),  # Define explícitamente la capa de entrada
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.summary()

TensorFlow version: 2.18.0
Eager execution: True


W0000 00:00:1738756085.380430   12188 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 221 (884.00 B)

 Trainable params: 221 (884.00 B)

 Non-trainable params: 0 (0.00 B)

## 2. Definir la estructura de los archivos de entrada

**training_data.csv**:
- Columna 1: strain_id
- Columnas 2 a 11: Para cada antibiótico (FEP, MER, IMI, AZT, CIP):
      Ejemplo:
        FEP_MIC, FEP_eval, MER_MIC, MER_eval, IMI_MIC, IMI_eval, AZT_MIC, AZT_eval, CIP_MIC, CIP_eval
  Las evaluaciones tienen 3 categorías: "sensitive", "resistant" e "indeterminate".
- Columnas 12 en adelante: Información de mutaciones para 71 genes.
   * Si no hay información para un gen: "-"
   * Si el gen es wild type: NaN
   * Si hay mutaciones: cadena con las mutaciones separadas por comas (ej. "A123T,V456G")

**test_data.csv**:
- Columna 1: strain_id
- Columnas 2 en adelante: Información de mutaciones para los mismos 71 genes.



## 3. Cargar y preprocesar los datos

In [3]:
# ## 3. Cargar y preprocesar los datos
# Leer el archivo de entrenamiento (1200 cepas)
train_df = pd.read_csv('input/training_data.csv', sep=";").reset_index(drop=True)
print("Dimensiones del DataFrame original:", train_df.shape)

# Definir nombres de columnas:
# ["strain_id", "FEP_MIC", "FEP_eval", "MER_MIC", "MER_eval", "IMI_MIC", "IMI_eval",
#  "AZT_MIC", "AZT_eval", "CIP_MIC", "CIP_eval", gene1, gene2, ..., gene71]
id_col = "strain_id"

# Evaluaciones clínicas de los antibióticos
eval_cols = ["FEP_eval", "MER_eval", "IMI_eval", "AZT_eval", "CIP_eval"]

# Columnas con genes de resistencia
gene_cols = train_df.loc[:, "PA0004":"PA5493"]

# Preprocesamiento de los genes: codificación multi-hot por gen
# La idea es, para cada gen, construir un vocabulario de mutaciones (observadas en el set de entrenamiento)
# y, para cada celda, crear un vector binario indicando la presencia de cada mutación.
def build_gene_vocab(df, gene_columns):
    vocab = {}
    for col in gene_columns:
        mutations = set()
        for val in df[col]:
            if pd.isna(val) or str(val).strip() == "-" or str(val).strip() == "":
                continue
            # Dividir las mutaciones por coma
            for mut in val.split(","):
                mut = mut.strip()
                if mut:
                    mutations.add(mut)
        # Ordenamos el vocabulario para tener un orden fijo
        vocab[col] = sorted(list(mutations))
    return vocab

gene_vocab = build_gene_vocab(train_df, gene_cols)

print('Vocabulario de mutaciones: gene_vocab')
print(gene_vocab)

Dimensiones del DataFrame original: (1045, 79)
Vocabulario de mutaciones: gene_vocab
{'PA0004': ['A623T', 'A627V', 'A720V', 'D4N', 'E168D', 'E215D', 'E239Q', 'E468D', 'G481D', 'H148N', 'H148N; aa467Δ2', 'H170Y', 'I529M', 'I529V', 'I769V', 'N5K', 'N5K; E468D', 'P749S', 'P789L', 'R22C', 'S466F', 'S466F; E721K', 'S466Y', 'S578A', 'T235S', 'T595I', 'T640S', 'V146L'], 'PA0005': ['A18S', 'I244L', 'I244T', 'I33F', 'N46S', 'Q45R', 'R62S', 'T55A', 'V10I', 'Y192F'], 'PA0424': ['A103G', 'A103G; V126E; nt373Δ1', 'A103T', 'C30R', 'K44M; V126E', 'L102P', 'L57Q', 'L95F', 'N79S', 'Q106R', 'Q49X', 'Q55P; V126E', 'R114C', 'R63H', 'R70Q', 'R73L; V126E', 'R78X', 'R82H', 'S92N', 'V126E', 'V126E; V132A', 'V126E; nt33Δ13', 'V126E; nt67Δ10', 'V132A', 'V132M', 'V42E', 'aa11∆4', 'aa90∆4', 'nt120Δ1', 'nt161InsC', 'nt215Δ11', 'nt217∆1', 'nt266InsG', 'nt318InsG', 'nt319Δ10', 'nt367Δ15', 'nt373Δ1', 'nt385InsGGGT', 'nt386InsGGGA', 'nt386InsGTGT', 'nt391InsGGGT', 'nt402∆11', 'nt47InsG', 'nt61Δ1', 'nt80InsA'], 'PA0425

## 3.1 Codificar matrices para el set de entrenamiento

In [4]:
# Función para codificar una fila: para cada gen, si la celda está vacía se retorna un vector de ceros,
# si tiene mutaciones, se asigna 1 en las posiciones correspondientes al vocabulario.
def encode_gene_row(row, gene_columns, vocab):
    features = []
    for col in gene_columns:
        gene_voc = vocab[col]
        vec = np.zeros(len(gene_voc), dtype=int)
        cell = row[col]
        if pd.isna(cell) or str(cell).strip() == "-" or str(cell).strip() == "":
            # Wild type: vector de ceros
            pass
        else:
            mutations = [m.strip() for m in cell.split(",") if m.strip() != ""]
            for mut in mutations:
                if mut in gene_voc:
                    idx = gene_voc.index(mut)
                    vec[idx] = 1
        # Agregar el vector para este gen a la lista de características
        features.extend(vec.tolist())
    return features

print("Número de índices únicos en train_df:", train_df.index.nunique())

# # Aplicar la codificación a cada fila para los genes en el set de entrenamiento
X_train_list = [encode_gene_row(row, gene_cols, gene_vocab) for _, row in train_df.iterrows()]
print("Número de muestras generadas:", len(X_train_list))
X_train_genes = np.array(X_train_list)
print("Dimensiones de X_train_genes:", X_train_genes.shape)

# Preparar las etiquetas para caracterizar clínicamente los valores MIC:
# Cada evaluación es una cadena con una de 3 categorías: "sensitive", "resistant" o "indeterminate".
y_antibiotics = {}
antibiotic_encoders = {}

for col in eval_cols:
    le = LabelEncoder()
    y_enc = le.fit_transform(train_df[col].astype(str).str.lower())
    antibiotic_encoders[col] = le
    # Convertimos a formato one-hot (aunque en clasificación binaria se puede usar una sola neurona con sigmoide)
    # One-hot encoding para 3 clases *********************************REVISAR**********************************¿como se codifican las 3 clases?
    y_antibiotics[col] = to_categorical(y_enc)


# Creamos el diccionario de salidas para el modelo multi-salida (5 salidas, una por antibiótico)
y_train = {
    'FEP': y_antibiotics["FEP_eval"],
    'MER': y_antibiotics["MER_eval"],
    'IMI': y_antibiotics["IMI_eval"],
    'AZT': y_antibiotics["AZT_eval"],
    'CIP': y_antibiotics["CIP_eval"]
}


# La entrada del modelo será únicamente la información codificada de los genes.
X_train_input = X_train_genes
print("Dimensiones de X_train_input:", X_train_input.shape)

# Cargar datos de test (200 cepas)
test_df = pd.read_csv('input/test_data.csv',sep=";")
# Se asume que test_data.csv tiene: "strain_id" y luego las 71 columnas de genes con los mismos nombres.
test_gene_cols = test_df.columns[1:]
X_test_genes = test_df.apply(lambda row: encode_gene_row(row, test_gene_cols, gene_vocab), axis=1)
X_test_input = np.array(X_test_genes.tolist())
print("Dimensiones de X_test_input:", X_test_input.shape)

Número de índices únicos en train_df: 1045
Número de muestras generadas: 1045
Dimensiones de X_train_genes: (1045, 2261)
Dimensiones de X_train_input: (1045, 2261)
Dimensiones de X_test_input: (44, 2261)


## 4. Definir una capa Cross para capturar interacciones entre mutaciones

In [7]:
# ## 4. Definir una capa Cross para capturar interacciones entre mutaciones
class CrossLayer(tf.keras.layers.Layer):
    def __init__(self, num_layers=1, **kwargs):
        super(CrossLayer, self).__init__(**kwargs)
        self.num_layers = num_layers

    def build(self, input_shape):
        self.cross_weights = []
        self.cross_biases = []
        d = int(input_shape[-1])
        for i in range(self.num_layers):
            self.cross_weights.append(
                self.add_weight(name=f'cross_w_{i}',
                                shape=(d, 1),
                                initializer='glorot_uniform',
                                trainable=True)
            )
            self.cross_biases.append(
                self.add_weight(name=f'cross_b_{i}',
                                shape=(d,),
                                initializer='zeros',
                                trainable=True)
            )
        super(CrossLayer, self).build(input_shape)

    def call(self, inputs):
        x0 = inputs
        x = inputs
        for i in range(self.num_layers):
            dot = tf.matmul(x, self.cross_weights[i])  # (batch_size, 1)
            x = x0 * dot + self.cross_biases[i] + x
        return x

    print("Matrices CrossLayer creadas")

Matrices CrossLayer creadas


## 5. Definir el modelo de redes neuronales multi-salida

In [8]:
# ## 5. Definir el modelo de redes neuronales multi-salida
#
# Se construirán 5 salidas, una para cada antibiótico, cada una con 3 neuronas (softmax).
input_layer = Input(shape=(X_train_input.shape[1],), name='input')
x = Dense(256, activation='relu')(input_layer)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
# Capa Cross para modelar interacciones entre las mutaciones
x = CrossLayer(num_layers=1)(x)
x = Dense(64, activation='relu')(x)

print("Salidas creadas")

# Salidas para cada antibiótico: FEP, MER, IMI, AZT, CIP
out_FEP = Dense(3, activation='softmax', name='FEP')(x)
out_MER = Dense(3, activation='softmax', name='MER')(x)
out_IMI = Dense(3, activation='softmax', name='IMI')(x)
out_AZT = Dense(3, activation='softmax', name='AZT')(x)
out_CIP = Dense(3, activation='softmax', name='CIP')(x)

print("Ejecutando modelo")

model = tf.keras.models.Model(inputs=input_layer, outputs=[out_FEP, out_MER, out_IMI, out_AZT, out_CIP])
model.compile(optimizer='adam',
              loss={'FEP': 'categorical_crossentropy',
                    'MER': 'categorical_crossentropy',
                    'IMI': 'categorical_crossentropy',
                    'AZT': 'categorical_crossentropy',
                    'CIP': 'categorical_crossentropy'},
              metrics={'FEP': 'accuracy',
                       'MER': 'accuracy',
                       'IMI': 'accuracy',
                       'AZT': 'accuracy',
                       'CIP': 'accuracy'})

model.summary()
print("Calculados parametros para modelo")
print(pd.Timestamp.now())

for col in eval_cols:
    unique_vals = train_df[col].astype(str).str.lower().unique()
    print(f"{col} -> {unique_vals}")

Salidas creadas
Ejecutando modelo


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 2261)      │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │    579,072 │ input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │     32,896 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cross_layer         │ (None, 128)       │        256 │ dropout_1[0][0]   │
│ (CrossLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │      8,256 │ cross_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ FEP (Dense)         │ (None, 3)         │        195 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ MER (Dense)         │ (None, 3)         │        195 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ IMI (Dense)         │ (None, 3)         │        195 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ AZT (Dense)         │ (None, 3)         │        195 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ CIP (Dense)         │ (None, 3)         │        195 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 621,455 (2.37 MB)

 Trainable params: 621,455 (2.37 MB)

 Non-trainable params: 0 (0.00 B)

Calculados parametros para modelo
2025-02-05 12:49:15.342407
FEP_eval -> ['s' 'r' 'i']
MER_eval -> ['s' 'r' 'i']
IMI_eval -> ['s' 'r' 'i']
AZT_eval -> ['i' 'r' 's']
CIP_eval -> ['r' 's' 'i']


## 6. Entrenar el modelo

In [9]:
# ## 6. Entrenar el modelo
print("Dimensiones del DataFrame original:", train_df.shape)
print("Dimensiones de X_train_input:", X_train_input.shape)
for key in y_train:
    print(f"Dimensiones de y_train[{key}]:", y_train[key].shape)

history = model.fit(X_train_input, y_train,
                    epochs=50, batch_size=32, validation_split=0.2)

print("Modelo entrenado")

Dimensiones del DataFrame original: (1045, 79)
Dimensiones de X_train_input: (1045, 2261)
Dimensiones de y_train[FEP]: (1045, 3)
Dimensiones de y_train[MER]: (1045, 3)
Dimensiones de y_train[IMI]: (1045, 3)
Dimensiones de y_train[AZT]: (1045, 3)
Dimensiones de y_train[CIP]: (1045, 3)
Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - AZT_accuracy: 0.5528 - AZT_loss: 1.0534 - CIP_accuracy: 0.4546 - CIP_loss: 0.9820 - FEP_accuracy: 0.4956 - FEP_loss: 1.0447 - IMI_accuracy: 0.3921 - IMI_loss: 1.0934 - MER_accuracy: 0.3603 - MER_loss: 1.0847 - loss: 5.2608 - val_AZT_accuracy: 0.1531 - val_AZT_loss: 1.3804 - val_CIP_accuracy: 0.3732 - val_CIP_loss: 0.8360 - val_FEP_accuracy: 0.9139 - val_FEP_loss: 0.7710 - val_IMI_accuracy: 0.5024 - val_IMI_loss: 0.9607 - val_MER_accuracy: 0.2201 - val_MER_loss: 1.0213 - val_loss: 4.9454
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - AZT_accuracy: 0.6470 - AZT_loss: 0.8563 - CIP_accuracy: 0.6490 - CIP_loss: 0.5981 - FEP_accuracy: 0.7611 - FEP_loss: 0

## 7. Predicción en el set de test

In [10]:
# ## 7. Predicción en el set de test
#
# En el set de test se dispone únicamente de la información de mutaciones (no de MIC).
predictions = model.predict(X_test_input)

# Para cada antibiótico, se toma la clase con mayor probabilidad.
pred_FEP = np.argmax(predictions[0], axis=1)
pred_MER = np.argmax(predictions[1], axis=1)
pred_IMI = np.argmax(predictions[2], axis=1)
pred_AZT = np.argmax(predictions[3], axis=1)
pred_CIP = np.argmax(predictions[4], axis=1)

# Convertir las predicciones a etiquetas legibles usando los encoders definidos para cada evaluación.
pred_FEP_labels = antibiotic_encoders["FEP_eval"].inverse_transform(pred_FEP)
pred_MER_labels = antibiotic_encoders["MER_eval"].inverse_transform(pred_MER)
pred_IMI_labels = antibiotic_encoders["IMI_eval"].inverse_transform(pred_IMI)
pred_AZT_labels = antibiotic_encoders["AZT_eval"].inverse_transform(pred_AZT)
pred_CIP_labels = antibiotic_encoders["CIP_eval"].inverse_transform(pred_CIP)

results_df = pd.read_csv('input/test_solved.csv', sep=";")

success = {"FEP":0, "MER":0, "IMI":0, "AZT":0, "CIP":0}
for index, result in results_df["FEP_eval"].items():
    if pred_FEP_labels[index] == result.lower():
        success["FEP"] = success["FEP"] + 1 

for index, result in results_df["MER_eval"].items():
    if pred_FEP_labels[index] == result.lower():
        success["MER"] = success["MER"] + 1 

for index, result in results_df["IMI_eval"].items():
    if pred_FEP_labels[index] == result.lower():
        success["IMI"] = success["IMI"] + 1 

for index, result in results_df["AZT_eval"].items():
    if pred_FEP_labels[index] == result.lower():
        success["AZT"] = success["AZT"] + 1 

for index, result in results_df["CIP_eval"].items():
    if pred_FEP_labels[index] == result.lower():
        success["CIP"] = success["CIP"] + 1 
        
print("Tasa de predicción correcta:")
for ab, value in success.items():
    print(ab, " : ", value/44)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Tasa de predicción correcta:
FEP  :  0.9090909090909091
MER  :  0.7954545454545454
IMI  :  0.8181818181818182
AZT  :  0.9318181818181818
CIP  :  0.7954545454545454
